# System-wide Night Log

In [ ]:
# Parameters. Set defaults here.
# Times Square replaces this cell with the user's parameters.

# day_obs values: TODAY, YESTERDAY, YYYY-MM-DD
# Report on observing nights that start upto but not included this day.
day_obs = '2024-09-04' # Change to 'TODAY' to test with default before push  

# Total number of days of data to display (ending on day_obs)
number_of_days = '1'  

In [ ]:
# Only use packages available in the Rubin Science Platform
import requests
from collections import defaultdict
import pandas as pd
from pprint import pp
from urllib.parse import urlencode
from IPython.display import display, Markdown, display_markdown
from matplotlib import pyplot as plt
import os
from datetime import datetime, date, timedelta
#! from rubin_scheduler.site_models import Almanac

In [ ]:
# Normalize Parameters (both explicit Times Squares params, in implicit ones)
limit = 99  # YAGNI: Auto get more if this isn't enough to get all requested DAYS

match day_obs.lower():
    case 'today':
        date = datetime.now().date()
    case 'yesterday':
        date = datetime.now().date()-timedelta(days=1)
    case _:
        date = datetime.strptime(day_obs, '%Y-%m-%d').date()
# date:  is EXLUSIVE (upto, but not including)
days = int(number_of_days)

# Thus: [min_day_obs,max_day_obs)
min_day_obs = (date - timedelta(days=days-1)).strftime('%Y%m%d') # Inclusive
max_day_obs = (date + timedelta(days=1)).strftime('%Y%m%d') # prep for Exclusive

response_timeout = 3.05  # seconds, how long to wait for connection
read_timeout = 20  # seconds
timeout = (float(response_timeout), float(read_timeout))

In [ ]:
# Set default env to "usdf" and try before PUSH to repo.
summit = 'https://summit-lsp.lsst.codes'
usdf = 'https://usdf-rsp-dev.slac.stanford.edu'
tucson = 'https://tucson-teststand.lsst.codes'

# Use server=tucson for dev testing.
# Use server=usdf to test before push.
server = os.environ.get('EXTERNAL_INSTANCE_URL', summit)

In [ ]:
# TODO For Times Square, UNCOMMENT next line and COMMENT the one after.
!pip install git+https://github.com/lsst-ts/ts_logging_and_reporting.git@prototype >/dev/null
#!pip install --upgrade ..
import lsst.ts.logging_and_reporting.source_adapters as sad
#from lsst.ts.logging_and_reporting.source_adapters import ExposurelogAdapter, NarrativelogAdapter

In [ ]:
try:
    import lsst.ts.logging_and_reporting.version
    lrversion = lsst.ts.logging_and_reporting.version.__version__
except:
    lrversion = 'LIVE'

try:
    from lsst_efd_client import EfdClient
    enable_efd = True
except:
    enable_efd = False

In [ ]:
display(Markdown(f'''
Report for **{date}** covering the previous **{days}** observing night(s).
- Run on logs from **{server}/**
- Using *Prototype* Logging and Reporting Version: **{lrversion}**
- {enable_efd=}
'''))

# Almanac

In [ ]:
alm = sad.Almanac()
almtab = f'''\
| Attribute | Value |
| --------- | ----- |
| Moon Rise | {alm.moon_rise_time.iso} |
| Moon Set  | {alm.moon_set_time.iso} |
| Twilight Morning  | {alm.twilight_morning.iso} |
| Twilight Evening  | {alm.twilight_evening.iso} |
'''
display(Markdown(almtab))

# Exposure Log

In [ ]:
exposure_adapter = sad.ExposurelogAdapter(server_url=server)
exposure_url = exposure_adapter.source_url
try:
    exposure_recs,url = exposure_adapter.get_messages(
                                        limit=limit,
                                        min_day_obs=min_day_obs,
                                        max_day_obs=max_day_obs,
                                        )
except Exception as err:
    exposure_recs = []
    msg = f'ERROR getting records from {url=}: {err=}'
    raise Exception(msg)

In [ ]:
print(f'Retrieved {len(exposure_recs)} records from {exposure_url}')

In [ ]:
if exposure_recs:
    tabstr = exposure_adapter.day_table(exposure_recs, 'date_added', time_only=False)
    #display(Markdown(tabstr))
    print(tabstr)

In [ ]:
display(Markdown(f"### <font color='red'>{exposure_url}/exposures/ Not yet functional on USDF</font>"))
gaps = exposure_adapter.get_observation_gaps()
if gaps:
    display(Markdown(f'### Date vs Observation Gap (minutes) for all Instruments'))
    for instrument, day_gaps in gaps.items():
        if len(day_gaps) == 0:
            display(Markdown(f'**No day gaps found for *{instrument=!s}* **'))
        else:
            x,y = zip(*day_gaps.items())
            df = pd.DataFrame(dict(day=x,minutes=y))
            df.plot.bar(x='day', y='minutes', title=f'{instrument=!s}')
else:
    print(f'No Observation Gaps found in exposures.')

# Narrative Log


In [ ]:
narrative_adapter = sad.NarrativelogAdapter(server_url=server)
narrative_url = narrative_adapter.source_url
try:
    # date like '2000-01-02 12:00:00'
    # str(datetime(2000, 1, 2, 12, 0, 0))
    min_date = str(datetime.strptime(min_day_obs,'%Y%m%d'))
    max_date = str(datetime.strptime(max_day_obs,'%Y%m%d'))
    print(f'Get data from {narrative_url}: {min_date} to {max_date}')
    narrative_recs,url = narrative_adapter.get_messages(
        limit=limit,
        min_date_end=min_date,
        max_date_end=max_date
    )
except Exception as err:
    narrative_recs = []
    msg = f'ERROR getting records from {url}: {err=}'
    raise Exception(msg)

print(f'Retrieved {len(narrative_recs)} records.')

In [ ]:
if narrative_recs:
    tabstr = narrative_adapter.day_table(narrative_recs, 'date_added')
    #display(Markdown(tabstr)))
    print(tabstr)